In [ ]:
import pandas as pd
import re
import numpy as np
# Importar os módulos do Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pyautogui
from selenium.common.exceptions import NoSuchElementException
import requests
planilha = pd.read_excel("all.xlsx")
insta_message = []
face_message = []
others_websites = []
find_instagram = []
notfind_instagram = []


##### Filtros

In [ ]:
planilha = planilha[~planilha['Place Name'].str.contains('Magazine|Kalunga|renner|Americanas|Americana|americanas|americana|Mercado Livre|mercado livre|shopee|Shopee|Renner', na=False)]
planilha = planilha.drop_duplicates(subset='Place Name')

##### Identificar se ele possui Instagram/Facebook/Site

In [ ]:
#Instagram
for i in range(len(planilha)):
    first_row = planilha.iloc[i]
    url = re.search(r'(www\.instagram\.com)', str(first_row["Website"]))

    if url:
        insta_message.append(first_row["Website"])
        planilha["Website"].replace(to_replace=first_row["Website"],value="Done",inplace=True)

    else:
        pass
#Facebook
for i in range(len(planilha)):
    first_row = planilha.iloc[i]
    url = re.search(r'(www\.facebook\.com)', str(first_row["Website"]))

    if url:
        face_message.append(first_row["Website"])
        planilha["Website"].replace(to_replace=first_row["Website"],value="Done",inplace=True)

    else:
        pass

#Other Websites
for i in range(len(planilha)):
    first_row = planilha.iloc[i]
    url2 = re.search(r'.www.', str(first_row["Website"]))

    if url2:
        others_websites.append(first_row["Website"])
        planilha["Website"].replace(to_replace=first_row["Website"],value="Done",inplace=True)
    else:
        pass

#### verificar se os sites possuem instagram

In [ ]:
for g in others_websites:
    try:
        response = requests.get(g)

        if response.status_code == 200:
            website = response.text
            
            url = re.search(r'www\.instagram\.com\/[A-Za-z0-9]+\/', website)

            if url:
                find_instagram.append(url)
                #planilha["Website"].replace(to_replace=first_row["Website"],value="Done",inplace=True)

            else:
                notfind_instagram.append(g)
                pass    
        else:
            print("Error: Could not retrieve website", g)
    except:
        print("critical error", g)

In [ ]:
for h in find_instagram:
    url = re.search(r'www\.instagram\.com\/[A-Za-z0-9]+\/', str(h))

    if url:
        matched_string = url.group()
        insta_message.append(matched_string)
    else:
        print("error")

##### Nomes e numeros

In [ ]:
list_numbers = []
for i in range(len(planilha)):
    first_row = planilha.iloc[i]
    if not pd.isnull(first_row["Phone"]):
        list_numbers.append(first_row["Phone"])
        planilha["Phone"].replace(to_replace=first_row["Phone"],value="Done",inplace=True)

Filtragem dos numeros

In [ ]:
list_numbers = [re.sub(r'[^\d]+', '', string) for string in list_numbers]
list_numbers = [item for item in list_numbers if item]

##### Verificar todos os sites que não funcionou nem o numero ou o Site

In [ ]:
nada_encontrado = []
for i in range(len(planilha)):
    first_row = planilha.iloc[i]
    if first_row["Phone"] != "Done":
        if first_row["Website"] != "Done":
            nada_encontrado.append(first_row["Place Name"])
            #colocar em um dicionario com as principais informações -> fazer alguma coisa para achar esses locais


# Processos

Enviar mensagens Instagram

In [ ]:
#base - Entrar no Instagram

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)

# Abrir o site do Instagram e fazer login
driver.get("https://www.instagram.com/")
time.sleep(4)
driver.find_element(By.NAME, "username").send_keys("COLOCAR SEU LOGIN")

driver.find_element(By.NAME, "password").send_keys("COLOCAR SUA SENHA")

driver.find_element(By.XPATH, '//button[@type="submit"]').click()
time.sleep(5)
try:
    driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div/div/div/button').click()
    time.sleep(5)
except:
    pass
driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]/button[2]').click()
time.sleep(3)
print("bora bora")
for j in range(len(insta_message)):
    print(j)
    try:
        driver.get(insta_message[j])
        time.sleep(3)
    except: #terminar essa parte
        location = pyautogui.locateOnScreen("reload.png", grayscale=False, confidence=.7)
        pyautogui.moveTo(location)
        pyautogui.moveRel(80, 0)
        pyautogui.click()
        #add control a in other way
        pyautogui.hotkey('delete')
        time.sleep(1)
        pyautogui.typewrite(insta_message[j])
        pyautogui.hotkey("enter")
        time.sleep(4)       
    try:
        try:
            location = pyautogui.locateOnScreen("follow.png", grayscale=False, confidence=.7)
            pyautogui.moveTo(location)
            pyautogui.click()
            time.sleep(1)
        except:
            pass
        try:
            location2 = pyautogui.locateOnScreen("message.png", grayscale=False, confidence=.7)
            pyautogui.moveTo(location2)
            pyautogui.click()
            time.sleep(5)
            try:
                # mensagem personalizada para cada loja
                name = re.search(r'\/([a-zA-Z0-9_]+)\/?$', insta_message[j])
                if name:
                    name = name.group(1)
                name = "Ola, " + str(name) + ", " + "Mensagem"
            except:
                name = "Ola, "+ "Mensagem"

            #find first the local for click and type
            pyautogui.typewrite(name)
            #pyautogui.hotkey('enter')
            time.sleep(2)
        except:
            pass

    except:
        print("erro", j)
        pass

    

Enviar mensagens WhatsApp

In [ ]:
#identifcar onde está o google
location = pyautogui.locateOnScreen("chrome.png", grayscale=False, confidence=.7)
pyautogui.moveTo(location)
pyautogui.click()
time.sleep(2)
for i in range(len(list_numbers)):
    print(i)
    #identifcar onde abrir a nova aba
    location = pyautogui.locateOnScreen("new window.png", grayscale=False, confidence=.7)
    pyautogui.moveTo(location)
    pyautogui.click()
    time.sleep(2)
    #Escrever o https://wa.me/55 + o numero da empresa
    pyautogui.typewrite("https://wa.me/55" + str(list_numbers[i]))
    time.sleep(1)
    pyautogui.hotkey("enter")
    time.sleep(2)
    #Clicar em continue to chat
    location = pyautogui.locateOnScreen("continue to chat.png", grayscale=False, confidence=.7)
    pyautogui.moveTo(location)
    pyautogui.click()
    time.sleep(2)
    location = pyautogui.locateOnScreen("iniciate message.png", grayscale=False, confidence=.7)
    pyautogui.moveTo(location)
    pyautogui.click()
    time.sleep(7)
    pyautogui.typewrite("ola amigo")
    #pyautogui.hotkey("enter")
    time.sleep(1)
    #except:
        #print(i, "algum erro aconteceu")